In [2]:
import warnings
warnings.filterwarnings('ignore')

import random
import keras
import os, glob
import librosa, librosa.display

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix

from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential, model_from_json
from keras import backend as K

from imblearn.over_sampling import RandomOverSampler



%load_ext autoreload
%autoreload 2

import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.model import CNN, CNN_nodropout

from settings import *

2023-05-22 06:49:44.096189: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 06:49:44.217516: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-22 06:49:44.898334: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-05-22 06:49:44.898407: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64]

In [8]:
cnn = CNN((128, 107))
cnn.load_model("baseline_results")

dataset_piano = pd.read_pickle(os.path.join(METADATA_DIR_RAW, 'data_violin.pkl'))
size = 107
dataset_piano['spectrogram'] = dataset_piano['spectrogram'].apply(lambda x: np.pad(x, ((0, 0), (0, size-x.shape[1])), 'constant'))

test_data = dataset_piano

print("Number of testing samples is", len(test_data))
print(test_data[:10])

X_test = test_data['spectrogram']
X_test = np.array([np.pad(x, ((0, 0), (0, size-x.shape[1])), 'constant') for x in dataset_piano['spectrogram']])
y_test = test_data['class_ID']

X_test = np.array([x.reshape( (128, size, 1) ) for x in X_test])

y_test_values=y_test
y_test = np.array(keras.utils.to_categorical(y_test, 10))

score = cnn.model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test precision:', score[2])
print('Test recall:', score[3])
print('Test f1-score:', score[4])

src.model - INFO - Initializing CNN
src.model - INFO - Input shape = (128, 107, 1)
src.model - INFO - CNN Initialized
src.model - INFO - Loading saved model
src.model - INFO - Model loaded from /home/tzag/danigil/dl/guitarCR/models
Number of testing samples is 100
     file_path file_name class_name  class_ID  \
0   em/em8.wav   em8.wav         em         7   
1   em/em6.wav   em6.wav         em         7   
2   em/em7.wav   em7.wav         em         7   
3   em/em3.wav   em3.wav         em         7   
4   em/em5.wav   em5.wav         em         7   
5   em/em1.wav   em1.wav         em         7   
6   em/em9.wav   em9.wav         em         7   
7   em/em4.wav   em4.wav         em         7   
8  em/em10.wav  em10.wav         em         7   
9   em/em2.wav   em2.wav         em         7   

                                                   y     sr  \
0  [0.026554763, 0.039172687, 0.034024365, 0.0322...  22050   
1  [-0.00038409373, -0.0042537, -0.00555648, 0.00...  22050   
2  [0.

In [9]:
size = 213
cnn = CNN_nodropout((128, size))
cnn.load_model("mymodel")

dataset_piano = pd.read_pickle(os.path.join(METADATA_DIR_RAW_NORMALIZED, 'data_violin.pkl'))
#dataset_piano['spectrogram'] = dataset_piano['spectrogram'].apply(lambda x: np.pad(x, ((0, 0), (0, size-x.shape[1])), 'constant'))

test_data = dataset_piano

print("Number of testing samples is", len(test_data))
print(test_data[:10])

X_test = test_data['spectrogram']
X_test = np.array([np.pad(x, ((0, 0), (0, size-x.shape[1])), 'constant') for x in dataset_piano['spectrogram']])
y_test = test_data['class_ID']

X_test = np.array([x.reshape( (128, size, 1) ) for x in X_test])

y_test_values=y_test
y_test = np.array(keras.utils.to_categorical(y_test, 10))

score = cnn.model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test precision:', score[2])
print('Test recall:', score[3])
print('Test f1-score:', score[4])

src.model - INFO - Input shape = (128, 213, 1)
src.model - INFO - Loading saved model
src.model - INFO - Model loaded from /home/tzag/danigil/dl/guitarCR/models
Number of testing samples is 100
     file_path file_name class_name  class_ID  \
0   em/em8.wav   em8.wav         em         7   
1   em/em6.wav   em6.wav         em         7   
2   em/em7.wav   em7.wav         em         7   
3   em/em3.wav   em3.wav         em         7   
4   em/em5.wav   em5.wav         em         7   
5   em/em1.wav   em1.wav         em         7   
6   em/em9.wav   em9.wav         em         7   
7   em/em4.wav   em4.wav         em         7   
8  em/em10.wav  em10.wav         em         7   
9   em/em2.wav   em2.wav         em         7   

                                                   y     sr  \
0  [0.06301807, 0.061424017, 0.06360255, 0.066206...  44100   
1  [0.0032537961, -0.022001859, -0.015184382, -0....  44100   
2  [0.011664668, 0.017987572, 0.00032704676, 0.00...  44100   
3  [0.01343386

In [13]:
dataset_piano = pd.read_pickle(os.path.join(METADATA_DIR_RAW, 'data_piano.pkl'))

from itertools import groupby

def all_equal(iterable):
    g = groupby(iterable)
    return next(g, True) and not next(g, False)

assert all_equal(dataset_piano['spectrogram'].map(lambda x: x.shape[0]))

max_size = max(dataset_piano['spectrogram'].map(lambda x: x.shape[1]))

In [16]:
dataset_piano['spectrogram'] = dataset_piano['spectrogram'].apply(lambda x: np.pad(x, ((0, 0), (0, max_size-x.shape[1])), 'constant'))

In [17]:
print([x.shape for x in dataset_piano['spectrogram']])

[(128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87)

In [6]:
dataset_guitar = pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED, 'data_guitar.pkl'))
print([x.shape for x in dataset_guitar['spectrogram']])

[(128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87)

In [24]:
dataset = pd.read_pickle(os.path.join(METADATA_DIR_RAW, 'data_piano.pkl'))
print([np.pad(x, ((0, 0), (0, 87-x.shape[1])), 'constant').shape for x in dataset['spectrogram']])

[(128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87), (128, 87)

In [25]:
dataset = pd.read_pickle(os.path.join(METADATA_DIR_RAW, 'data_piano.pkl'))

test_data = dataset

print("Number of testing samples is", len(test_data))
print(test_data[:10])

X_test = test_data['spectrogram']
X_test = np.array([np.pad(x, ((0, 0), (0, 87-x.shape[1])), 'constant') for x in dataset['spectrogram']])
y_test = test_data['class_ID']

X_test = np.array([x.reshape( (128, 87, 1) ) for x in X_test])

y_test_values=y_test
y_test = np.array(keras.utils.to_categorical(y_test, 10))

Number of testing samples is 100
     file_path file_name class_name  class_ID  \
0   em/em8.wav   em8.wav         em         7   
1   em/em6.wav   em6.wav         em         7   
2   em/em7.wav   em7.wav         em         7   
3   em/em3.wav   em3.wav         em         7   
4   em/em5.wav   em5.wav         em         7   
5   em/em1.wav   em1.wav         em         7   
6   em/em9.wav   em9.wav         em         7   
7   em/em4.wav   em4.wav         em         7   
8  em/em10.wav  em10.wav         em         7   
9   em/em2.wav   em2.wav         em         7   

                                                   y     sr  \
0  [-0.0026758127, -0.0040184762, -0.004359124, -...  22050   
1  [0.0007722303, 0.0018129349, 0.0006996244, 0.0...  22050   
2  [-0.0026981905, -0.0034450963, -0.0021259002, ...  22050   
3  [-0.002258081, -0.0025149286, -0.0010869578, -...  22050   
4  [-0.005968757, -0.0077354126, -0.006970454, -0...  22050   
5  [-0.0008049598, -0.0005307044, -0.00051933434,

In [18]:
def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    '''Calculates the F score, the weighted harmonic mean of precision and recall.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    '''
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
        
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0.0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    '''Calculates the f-measure, the harmonic mean of precision and recall.
    '''
    return fbeta_score(y_true, y_pred, beta=1)

In [26]:
with open(MODEL_1_JSON, "r") as json_file:
	loaded_model_json = json_file.read()

model = model_from_json(loaded_model_json)
model.load_weights(MODEL_1_H5)

model.compile(
            optimizer="Adam",
            loss="categorical_crossentropy",
            metrics=['accuracy', precision, recall, fmeasure])

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test precision:', score[2])
print('Test recall:', score[3])
print('Test f1-score:', score[4])

4/4 [==============================] - 0s 16ms/step - loss: 24.2755 - accuracy: 0.2000 - precision: 0.2219 - recall: 0.2109 - fmeasure: 0.2161
Test loss: 24.275524139404297
Test accuracy: 0.20000000298023224
Test precision: 0.22194939851760864
Test recall: 0.2109375
Test f1-score: 0.2161121815443039
